# Purpose:

Notebook to show how to use feature importance and signatures within Open Supernova Catalog

It uses the coniferest package https://github.com/snad-space/coniferest

Data available from https://github.com/snad-space/snad

See https://arxiv.org/abs/1905.11516 for further detailed information about the sample and identified anomalies within


# 0) Environment

Please modify the *prefix* variable in order to point to your data

In [ ]:
prefix='.'
%ls $prefix/snad-space/snad

# 1) Preparation & loading data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd

In [ ]:
# specific imports
from coniferest.pineforest import PineForest
from coniferest.pariou import comp_signature, n_expected_visits

In [ ]:
# getting the data
data_file_1=os.path.join(prefix,'snad-space/snad/data/extrapol_-20.0_100.0_B,R,I.csv')
data_file_2=os.path.join(prefix,'snad-space/snad/data/extrapol_-20.0_100.0_g,r,i.csv')
data_file_3=os.path.join(prefix,'snad-space/snad/data/extrapol_-20.0_100.0_g_pr,r_pr,i_pr.csv')

df_1=pd.read_csv(data_file_1)
df_2=pd.read_csv(data_file_2)
df_3=pd.read_csv(data_file_3)

In [ ]:
# for exploration : all keys are identical excapt the original band names (columns 1 to 3)
# [z for z in zip(df_1.keys(),df_2.keys(),df_3.keys())]

In [ ]:
# Build the data array
mask = df_1.columns.str.contains('^[gri][+-]') # all masks identical
data_1=df_1.loc[:,mask].to_numpy()
data_2=df_2.loc[:,mask].to_numpy()
data_3=df_3.loc[:,mask].to_numpy()
data=np.concatenate((data_1,data_2,data_3)).astype('float32')

In [ ]:
# data still have to be normalized
data=np.multiply(data,1/np.max(data,axis=1)[:,np.newaxis])
# note that this deviates from the paper, which adds the total flux as an additional feature

In [ ]:
# for reference, record object names
names=np.concatenate((df_1.Name.to_numpy(), df_2.Name.to_numpy(),df_3.Name.to_numpy()))

In [ ]:
# basic information about the sample
np.shape(data)

## Propose a visualization

In [ ]:
# colormaps

from matplotlib.colors import ListedColormap
N = 256
vals_g = np.ones((N, 4))
vals_r = np.ones((N, 4))
vals_y = np.ones((N, 4))
# g
vals_g[:, 0] = np.linspace(0/256, 231/256, N)
vals_g[:, 1] = np.linspace(64/256, 256/256, N)
vals_g[:, 2] = np.linspace(0/256, 231/256, N)
cmp_g = ListedColormap(vals_g)
# r
vals_r[:, 0] = np.linspace(128/256, 256/256, N)
vals_r[:, 1] = np.linspace(0/256, 231/256, N)
vals_r[:, 2] = np.linspace(0/256, 231/256, N)
cmp_r = ListedColormap(vals_r)
# y
vals_y[:, 0] = np.linspace(178/256, 244/256, N)
vals_y[:, 1] = np.linspace(160/256, 244/256, N)
vals_y[:, 2] = np.linspace(0/256, 231/256, N)
cmp_y = ListedColormap(vals_y)


In [ ]:
# Main plotting method
def plot_data_n(data,n,color=None,ylabel="relative flux",title=None):
    
    _,length=np.shape(data)

    time=np.arange(-20,-20+length//3)

    if color is not None:

        plt.scatter(time,data[n,:length//3],marker='.',cmap=cmp_g,c=color[:length//3])
        plt.scatter(time,data[n,length//3:2*length//3],marker='.',cmap=cmp_r,c=color[length//3:2*length//3])
        plt.scatter(time,data[n,2*length//3:],marker='.',cmap=cmp_y,c=color[2*length//3:])
        
    else:
        plt.scatter(time,data[n,:length//3],cmap=cmp_g,c=np.zeros(length//3),vmax=1,s=14)
        plt.scatter(time,data[n,length//3:2*length//3],marker='^',cmap=cmp_r,c=np.zeros(length//3),vmax=1,s=12)
        plt.scatter(time,data[n,2*length//3:],marker='*',cmap=cmp_y,c=np.zeros(length//3),vmax=1,s=8)

    if title is None:
        plt.title(names[n])
    else:
        plt.title(title)
    plt.ylabel(ylabel)
    plt.xlabel("time (days)")

In [ ]:
# to plot averages and std of data
def plot_envelope(data):
    mean=np.mean(data,axis=0)
    std=np.std(data,axis=0)
    _,length=np.shape(data)
    time=np.arange(-20,-20+length//3)
    plt.plot(time,mean[:length//3],color=cmp_g(0.75))
    plt.fill_between(time,(mean-std)[:length//3],(mean+std)[:length//3],color=cmp_g(0.0),alpha=0.05)
    plt.plot(time,mean[length//3:2*length//3],color=cmp_r(0.75))
    plt.fill_between(time,(mean-std)[length//3:2*length//3],(mean+std)[length//3:2*length//3],color=cmp_r(0.0),alpha=0.05)
    plt.plot(time,mean[2*length//3:],color=cmp_y(0.75))
    plt.fill_between(time,(mean-std)[2*length//3:],(mean+std)[2*length//3:],color=cmp_y(0.0),alpha=0.05)

In [ ]:
# One example
plot_data_n(data,13)
# to plot the average, uncomment this line
#plot_envelope(data)

# 2) Build and analyze Isolation Forest

In [ ]:
print("Number of trees of depth 10 (trained with 1024 elements) needed to have 1 visit in average:",1/n_expected_visits(data))

In [ ]:
# fit and get scores (same code for PineForest and IsolationForest, but PineForest is faster)
pineforest = PineForest(n_trees=3000,n_subsamples=1024, weight_ratio=1)
pineforest.fit(data)
scores_pf = pineforest.score_samples(data)

In [ ]:
# score distribution
plt.hist(scores_pf,bins='fd');
plt.yscale('log')
plt.xlabel('score')
plt.ylabel('count');

## Plot the most outliers and their signatures

In [ ]:
maxiter=20
deltas=comp_signature(pineforest,data,np.argsort(scores_pf)[:maxiter])
for i in range (maxiter):
    plt.figure()
    plt.subplot(211)
    plot_data_n(data,np.argsort(scores_pf)[i],deltas[i])
    plot_envelope(data)
    plt.subplot(212)
    plot_data_n(deltas,i,ylabel='Signature')

## Plot the most normals and their signatures

In [ ]:
deltas=comp_signature(pineforest,data,np.argsort(scores_pf)[-maxiter:])
for i in range (maxiter):
    plt.figure()
    plt.subplot(211)
    plot_data_n(data,np.argsort(scores_pf)[-maxiter:][i],deltas[i])
    plot_envelope(data)
    plt.subplot(212)
    plot_data_n(deltas,i,ylabel='Signature')

## Plot the average signatures per identified anomalies

In [ ]:
slsn=["SDSS-II SN 17789", "SN2015bn", "PTF10aagc", "SN2213-1745"]
pecIa=["SN2016bln","PS15cfn", "SNLS-03D1cm","SN2002bj","SN2013cv"]
unusualII=["SN2013ej", "SN2016ija"]
agn=["SN2006kg"]
microlens=["Gaia16aye"]
stars=['SDSS-II SN 5314', 'SDSS-II SN 14170', 'SDSS-II SN 15565', 'SDSS-II SN 13725', 'SDSS-II SN 13741',
       'SDSS-II SN 19699', 'SDSS-II SN 18266', 'SDSS-II SN 4226', 'SDSS-II SN 2809', 'SDSS-II SN 6992']
qso=["SDSS-II SN 1706", "SDSS-II SN 17756", "SDSS-II SN 17339", "SDSS-II SN 17509", "SDSS-II SN 4652",
     "SDSS-II SN 19395"]

In [ ]:
def plot_average_signature(sublist):
    indices=[list(names).index(sn) for sn in sublist]
    sig,weights=comp_signature(pineforest,data,indices,full_output=True)
    avg_sig=np.sum(sig*weights,axis=0)/np.sum(weights,axis=0)
    plot_data_n(avg_sig[np.newaxis,:],0,ylabel='Signature',title="")

In [ ]:
plot_average_signature(slsn)

In [ ]:
plot_average_signature(pecIa)

In [ ]:
plot_average_signature(unusualII)

In [ ]:
plot_average_signature(agn)

In [ ]:
plot_average_signature(microlens)

In [ ]:
plot_average_signature(stars)

In [ ]:
plot_average_signature(qso)

## Plot the average for the full sample

In [ ]:
## And last : plot the full average for everyone
plot_average_signature(names)